In [1]:
import argparse
import time
import numpy as np
import torch
import random
from models import *
from utils.datasets import *
from utils.utils import *
import shutil
  # Reload the module


In [2]:
cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if cuda else 'cpu')

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
if cuda:
	torch.cuda.manual_seed(0)
	torch.cuda.manual_seed_all(0)
	torch.backends.cudnn.benchmark = True
print(device)

cuda:0


In [6]:
# epochs=5
# batch_size=4
# data_config_path=r'C:\Users\vrind\Downloads\segmentationnproject\YOLOv3-quadrangle\cfg\PHOTO.data'
# img_size=512*640
# resume=False
# cfg_path=r"C:\Users\vrind\Downloads\segmentationnproject\YOLOv3-quadrangle\cfg"
# model_cfg_path=r"C:\Users\vrind\Downloads\segmentationnproject\YOLOv3-quadrangle\cfg\yolov3.cfg"
epochs=5
batch_size=4
data_config_path=r'D:\personal\GiveToShri\cfg\PHOTO.data'
img_size=32*19
resume=False
cfg_path=r"D:\personal\GiveToShri\cfg"
model_cfg_path=r"D:\personal\GiveToShri\cfg\yolov3.cfg"



In [7]:
def main():
	os.makedirs('weights', exist_ok=True)

	# Configure run
	data_config = parse_data_config(data_config_path)
	num_classes = int(data_config['classes'])
	train_path = data_config['train']

	# Initialize model
	model = Darknet(model_cfg_path, img_size)

	# Get dataloader
	dataloader = load_images_and_labels(train_path, batch_size=batch_size, img_size=img_size, augment=True)

	# Reload saved optimizer state
	start_epoch = 0
	best_loss = float('inf')

	load_weights(model, 'weights/darknet53.conv.74')

	model.to(device).train()

	optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=.9, weight_decay=5e-4)

	#model_info(model)
	t0, t1 = time.time(), time.time()
	mean_recall, mean_precision = 0, 0
	print('%11s' * 12 % (
		'Epoch', 'Batch', 'conf', 'cls', 'loss', 'P', 'R', 'nTargets', 'TP', 'FP', 'FN', 'time'))
	for epoch in range(epochs):
		epoch += start_epoch

		# Update scheduler (manual)  at 0, 54, 61 epochs to 1e-3, 1e-4, 1e-5
		if epoch < 30:
			lr = 1e-4
		elif epoch < 60:
			lr = 1e-5
		else:
			lr = 1e-6
		for g in optimizer.param_groups:
			g['lr'] = lr

		ui = -1
		rloss = defaultdict(float)  # running loss
		metrics = torch.zeros(3, num_classes)
		optimizer.zero_grad()

		for i, (imgs, targets) in enumerate(dataloader):
			if sum([len(x) for x in targets]) < 1:  # if no targets continue
				continue

			# SGD burn-in
			if (epoch == 0) & (i <= 1000):
				lr = 1e-4 * (i / 1000) ** 4
				for g in optimizer.param_groups:
					g['lr'] = lr

			# Compute loss, compute gradient, update parameters
			loss = model(imgs.to(device), targets, requestPrecision=True)
			loss.backward()

			# accumulated_batches = 1  # accumulate gradient for 4 batches before stepping optimizer
			# if ((i+1) % accumulated_batches == 0) or (i == len(dataloader) - 1):
			optimizer.step()
			optimizer.zero_grad()

			# Compute running epoch-means of tracked metrics
			ui += 1
			metrics += model.losses['metrics']
			TP, FP, FN = metrics
			for key, val in model.losses.items():
				rloss[key] = (rloss[key] * ui + val) / (ui + 1)

			# Precision
			precision = TP / (TP + FP)
			k = (TP + FP) > 0
			if k.sum() > 0:
				mean_precision = precision[k].mean()

			# Recall
			recall = TP / (TP + FN)
			k = (TP + FN) > 0
			if k.sum() > 0:
				mean_recall = recall[k].mean()
			
			s = ('%11s%11s' + '%11.3g' * 10) % (
				'%g/%g' % (epoch, epochs - 1), '%g/%g' % (i, len(dataloader) - 1), rloss['conf'], rloss['cls'],
				rloss['loss'], mean_precision, mean_recall, model.losses['nT'], model.losses['TP'],
				model.losses['FP'], model.losses['FN'], time.time() - t1)
			t1 = time.time()
			print(s)

		# Write epoch results
		with open('results.txt', 'a') as file:
			file.write(s + '\n')

		# Update best loss
		loss_per_target = rloss['loss'] / rloss['nT']
		if loss_per_target < best_loss:
			best_loss = loss_per_target

		# Save latest checkpoint
		checkpoint = {'epoch': epoch,
					  'best_loss': best_loss,
					  'model': model.state_dict(),
					  'optimizer': optimizer.state_dict()}
		torch.save(checkpoint, 'weights/latest.pt')

		# Save best checkpoint

		if best_loss == loss_per_target:
			#os.system('cp weights/latest.pt weights/best.pt')
   			shutil.copy('weights/latest.pt', 'weights/best.pt')


		# Save backup weights every 5 epochs
		if (epoch > 0) & (epoch % 20 == 0):
			#os.system('cp weights/latest.pt weights/backup' + str(epoch) + '.pt')
			shutil.copy('weights/latest.pt', f'weights/backup{epoch}.pt')


	# Save final model
	dt = time.time() - t0
	print('Finished %g epochs in %.2fs (%.2fs/epoch)' % (epoch, dt, dt / (epoch + 1)))

In [8]:
main()

      Epoch      Batch       conf        cls       loss          P          R   nTargets         TP         FP         FN       time
        0/4        0/4       93.6          0        111          0          0         17          0   5.37e+04          3       3.18
        0/4        1/4       79.5          0       94.5          0          0         12          0   5.41e+04          1      0.271
        0/4        2/4       73.6          0       87.8          0          0         11          0   5.36e+04          1      0.363
        0/4        3/4       80.5          0         97          0          0         20          0    5.4e+04          5      0.383
        0/4        4/4       68.9          0         83          0          0          1          0   1.34e+04          0       1.61
        1/4        0/4       26.2          0       30.8          0          0          5          0   5.39e+04          2       1.16
        1/4        1/4       67.7          0       81.4          0   